In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium import webdriver
import time
from tqdm.notebook import tqdm # Progress bar

### Personajes jugables

En este juego los personajes jugables son dioses. Estos pueden tomar diferentes roles: asesino, guardian, cazador, mago o guerrero.

Las clases son, en esencia, distintas, pero suele ocurrir que un personaje pueda tomar varios de esos roles, aunque el juego le asigna uno por defecto.

Existen también algunos casos en los que un personaje tienen asignados un rol, según el juego, que no coincide con el verdadero uso que le dan los jugadores en una partida. Por ejemplo, aunque Martichoras tiene asignado el rol de cazador (Hunter), se usa mucho más en el rol de mago. Estos casos varían por ajustes del juego. Si quieres más información sobre algún personaje concreto, es posible encontrarla en este [enlace](https://www.smitefire.com/smite/gods), y solo hay que tener en cuenta algunas asociaciones:

+ Mid: mago.
+ ADC: cazador.
+ Support: guardian.
+ Solo: warrior.
+ Jungla: asesino.

Por tanto, una de las primeras guías de Martichoras será algo como "Conquest Mid Build", siendo Conquest el modo de juego más popular de Smite. Las recomendaciones, estarán principalmente sesgadas a lo recomendado para Conquest.

In [2]:
# Abrimos archivos para escritura
file_names = open('data/names.txt','w') # solo nombres
file_all = open("data/gods_names_roles.txt", "w") # nombres y roles

# Buscamos dioses por roles y los clasificamos con ellos
roles = ["Assassin", "Guardian", "Hunter", "Mage", "Warrior"]
for role in roles:
    url = f"https://smite.fandom.com/wiki/Category:{role}_gods"
    
    downloaded_text = requests.get(url, headers = {"User-agent": "your bot 0.1"}).text
    html = BeautifulSoup(downloaded_text)    
    
    for div in html.select("div.mw-category-group"):
        gods = div.find_all("a")
        
        for god in gods:
            # Cada fila es un dios con su role asignado
            name = god.text.strip()
            file_names.write(name+",")
            file_all.write(name + "," + role + "\n")
            
file_names.close()
file_all.close()

pd.read_csv("data/gods_names_roles.txt", header=None).head()

,0,1
0,Arachne,Assassin
1,Awilix,Assassin
2,Bakasura,Assassin
3,Bastet,Assassin
4,Camazotz,Assassin


### Scrapping de datos públicos de usuarios
Vamos a scrapear información de jugadores profesionales y de otros jugadores noveles:

+ Para encontrar a los profesionales, he buscado a los componentes de varios equipos competitivos de este [enlace](https://www.smiteproleague.com/teams/). También he incorporado a Hex Mambo, porque lo conocía.

+ Para los casuals, entre los que me incluyo, he cogido a algunos jugadores que han coincidido en partida conmigo.

Con los nombres de usuario he buscado en [smite.guru](https://smite.guru/) la información de cada jugador. Concretamente, he tomado el tiempo de juego con cada personaje, que será lo que utilizaré para medir la opinión del jugador.

Además, el juego lleva existiendo 10 años, y han variado muchas mecánicas y personajes desde su inicio. Uno de los cambios más importantes se produjo en la temporada anterior (Season 10), por lo que, en lugar de recabar información de todo el tiempo de juego, me centraré en lo que haya de la Season 10 en adelante (Seadon 10 y la actual Season 11).


In [3]:
usuarios = [   
    # Hex Mambo: 
    "2752037-Julio", "2175091-Hawkk", "2464007-Dzoni", "3434717-Spudio", "6799515-Cozyx",
    # Jade Dragons
    "1232115-Nika", "86523-LASBRA", "250322-Dardez", "4151583-PolarBearMike", "701305469-VaporishCoast",
    # Atlantis Leviathans
    "4057379-fineokay", "10083290-Sheento", "2784051-Adapting","10045885-Ronngyu", "1153439-PandaCat",
    # Camelot Kings
    "918206-Variety", "834980-CaptainTwig", "4111789-Theakz", "2022090-Quig", "10059174-Jarcorrr", "819265-Biggy",
    # Eldritch Hounds
    "2272206-Duck3y", "704101719-Oathhh", "5544740-BennyQ", "718039339-NeilMah", "700668575-Cutesypluto", "1722443-ElChuckles",
    # Gilded Gladiators
    "700114854-Cxnnahh", "3413550-Kyrmi", "702109726-Jangaru", "5008455-Awesomejake408", "730042-Snoopy", "4827135-KSier",
    # HIGHLAND RAVENS
    "825956-Haddix", "841456-Screammmmm", "1764272-Venenu", "1457496-Hurriwind", "367737-Zapman", "700115951-Marbach",
    # ONI WARRIORS
    "3390780-SoloOrTroll", "10050945-Panitom", "7780588-Pegon", "1134771-Genetics", "6422599-Netrioid", "10038700-Oxiledeon",
    # STYX FERRYMEN
    "1804371-Baskin", "3101009-Cyno", "360154-Paul", "40870-Aror", "144582-CycloneSpin", "307408-Realzx",
    # Otros usuarios (no expertos)
    "12057663-zonyiru", "717898767-TheFatKiing", "12410269-R3N3GAN", "716550989-guapiti", "10983503-L1mmbo", "719975272-AlexSomAndrew"
]

def scrap_season_info(usuarios, gods_names, seasons=[10, 11]):

    # Nombre del jugador: quitamos el código inicial
    players = []
    for user in usuarios:
        players.append(re.search(r"-\w+", user).group()[1:])

    # Horas de uso de cada personaje
    df = pd.DataFrame(columns=gods_names, index=players, data=0)
    for i in tqdm(range(len(players))):
        user = usuarios[i]
        name = players[i]
        for season in seasons:
            url = f"https://smite.guru/profile/{user}/champions?season={season}"
            downloaded_text = requests.get(url, headers = {"User-agent": "your bot 0.1"}).text
            html = BeautifulSoup(downloaded_text, "lxml")  # css selector
    
            # Información de cada personaje
            for div in html.select("div.row.champion-table__row"):
                current_god = div.find("div", "row__champion__name").text.strip()        
                played_time = div.find_all("div")[-3].text.strip()
                
                try:
                    horas = int(re.search(r"[0-9.,]+h", played_time).group()[:-1].replace(',', '')) #horas
                except:
                    horas = 0
                try:
                    minutos = int(re.search(r"\d+m", played_time).group()[:-1]) #minutos
                except:
                    minutos = 0
                
                df.loc[name, current_god] += (horas + minutos/60) 
            time.sleep(1) # to avoid being aggresive with petitions
        
    return df

In [4]:
result = scrap_season_info(usuarios, 
                           gods_names=pd.read_csv("data/names.txt", header=None).iloc[0,:].tolist())

  0%|          | 0/57 [00:00<?, ?it/s]

In [ ]:
result.to_csv("data/players_info__seasons10y11.csv")